In [1]:
from src.OPFInterface import OPFClassifier
from functools import cmp_to_key
import pandas as pd
import numpy as np
from os import listdir
from src.utils import error, read_df, euclidean_distance_classify
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, no_update
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
X, y, X_test, y_test, df_errors = read_df("MelbournePedestrian")

In [3]:
# from sklearn.manifold import MDS
# embedding = MDS(n_components=2)
# X_t = embedding.fit_transform(X)

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda_clf = LDA(n_components=2)
X_t = lda_clf.fit_transform(X, y)

In [5]:
palette = sns.color_palette("tab20")

fig = go.Figure(data=[
    go.Scatter(
        x=X_t[:,0],
        y=X_t[:,1],
        mode="markers",
        marker=dict(
            colorscale='viridis',
            color=[f'rgb{palette[y[num]%len(palette)]}' for num in range(len(X))],
            line={"color": "#444"},
            reversescale=True,
            sizeref=45,
            sizemode="diameter",
            opacity=0.6,
        )
    )
])


# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    plot_bgcolor='rgba(255,255,255,0.1)'
)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=np.array(range(len(X[num]))), y=X[num],
        line_color=f'rgb{palette[y[num]%len(palette)]}',
    ))
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.update_xaxes(visible=False, showticklabels=False)
    fig.update_layout(
        showlegend=False,
        plot_bgcolor="white",
        margin=dict(t=10,l=10,b=10,r=10),
        height=100,
        width=120
    )

    children = [
        html.Div(children=[
            html.H4(f"#{num}"),
            html.Div(f"Label {y[num]}"),
            dcc.Graph(figure=fig)
        ],
        style={'width': '120px', 'white-space': 'normal'})
    ]

    return True, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')

In [5]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from collections import OrderedDict

palette = sns.color_palette("tab20")

def get_edge_trace(G):
    edge_x = []
    edge_y = []

    #etext = [f'weight{w}' for w in list(nx.get_edge_attributes(G, 'diameter').values())]
    xtext=[]
    ytext=[]
    for edge in G.edges():

        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        xtext.append((x0+x1)/2)
        ytext.append((y0+y1)/2)
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        mode='lines'
    )
    eweights_trace = go.Scatter(x=xtext,y= ytext, mode='text',
                              marker_size=0.5,
                              text=[0.45, 0.7, 0.34],
                              textposition='top center',
                              hovertemplate='weight: %{text}<extra></extra>')
    return edge_trace, eweights_trace


def get_node_trace(G):
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker_size=12,
        #hoverinfo='text', #THIS LINE HAS NO SENSE BECAUSE text WAS NOT DEFINED
        marker_color='RoyalBlue',
        )

    return node_trace



tail = [1, 2, 3]
head = [2, 3, 4]

xpos = [0, 1, 2, 3]
ypos = [0, 0, 0, 0]
w = [1, 2, 3]
xpos_ypos = [(x, y) for x, y in zip(xpos, ypos)]

ed_ls = [(x, y) for x, y in zip(tail, head)]
G = nx.Graph()
G.add_edges_from(ed_ls)


pos = OrderedDict(zip(G.nodes, xpos_ypos))
edge_w = OrderedDict(zip(G.edges, w))
nx.set_node_attributes(G, pos, 'pos')
nx.set_edge_attributes(G, edge_w, 'weight')

# convert to plotly graph
edge_trace, eweights_trace = get_edge_trace(G)
node_trace = get_node_trace(G)

fig = go.Figure(data=[edge_trace, node_trace,eweights_trace ],
                layout=go.Layout(
                    title='<br>Network graph made with Python',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis_visible=False,
                    yaxis_visible=False)
                )



# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    plot_bgcolor='rgba(255,255,255,0.1)'
)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    
    pt = hoverData["points"][0]
    if pt["curveNumber"] != 2:
        return False, no_update, no_update
        
    
    bbox = pt["bbox"]
    num = pt["pointNumber"]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=np.array(range(len(X[num]))), y=X[num],
        line_color=f'rgb{palette[y[num]%len(palette)]}',
    ))
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.update_xaxes(visible=False, showticklabels=False)
    fig.update_layout(
        showlegend=False,
        plot_bgcolor="white",
        margin=dict(t=10,l=10,b=10,r=10),
        height=100,
        width=120
    )

    children = [
        html.Div(children=[
            html.H4(f"#{num}"),
            html.Div(f"Label {y[num]}"),
            dcc.Graph(figure=fig)
        ],
        style={'width': '120px', 'white-space': 'normal'})
    ]

    return True, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline')